In [1]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
input_folder = r'D:\ai\subhi\undistorted gear images'  # Raw images for model 1
model1_path = r'D:\ai\subhi\dante.pt'          # Teeth detection model
model2_path = r'D:\ai\subhi\best2.pt'                   # Circle detection model

output_root = r'D:\ai\subhi\predictions'  # Use Windows path here

model1_output_folder = os.path.join(output_root, 'model1_non_defective')
model2_annotated_folder = os.path.join(output_root, 'model2_annotated')
model2_circle_folder = os.path.join(model2_annotated_folder, 'circles')
log_path = os.path.join(output_root, "diameter_log.txt")

os.makedirs(model1_output_folder, exist_ok=True)
os.makedirs(model2_annotated_folder, exist_ok=True)
os.makedirs(model2_circle_folder, exist_ok=True)

PIXEL_TO_MM = 0.1972
EXPECTED_TEETH = 18
EXPECTED_DIAMETER_MM = 40.0
ERROR_CORRECTION_MM = 1.35
TOLERANCE_MM = 1.35

assert os.path.exists(model1_path), f"Model 1 weights missing: {model1_path}"
assert os.path.exists(model2_path), f"Model 2 weights missing: {model2_path}"

model1 = YOLO(model1_path)
model2 = YOLO(model2_path)

def label_teeth_defect(detected_teeth_count, expected_teeth):
    return detected_teeth_count == expected_teeth

def is_circle(cnt, threshold=0.7):
    perimeter = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * (area / (perimeter ** 2))
    return circularity > threshold

In [3]:
# --- Model 1: Teeth Detection ---
input_images = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not input_images:
    print(f"⚠️ No images found in input folder: {input_folder}")
else:
    print(f"ℹ️ Found {len(input_images)} images in input folder for Model 1.")

diameter_log = []
saved_non_defective_images = 0

print("=== Running Model 1: Teeth Detection ===")
for fname in input_images:
    img_path = os.path.join(input_folder, fname)
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Cannot read image {fname}, skipping.")
        continue

    results = model1(img_path)[0]

    detected_teeth = 0
    if results.boxes and results.boxes.xyxy is not None:
        detected_teeth = len(results.boxes.xyxy.cpu().numpy())

    is_defective = not label_teeth_defect(detected_teeth, EXPECTED_TEETH)

    if is_defective:
        print(f"❌ {fname}: Defective teeth count ({detected_teeth}) != expected ({EXPECTED_TEETH})")
        # Skip saving defective
        continue
    else:
        print(f"✅ {fname}: Teeth OK ({detected_teeth}) - saved for Model 2")
        save_path = os.path.join(model1_output_folder, fname)
        cv2.imwrite(save_path, img)
        saved_non_defective_images += 1

if saved_non_defective_images == 0:
    print(f"⚠️ No non-defective images found after Model 1 detection.")
else:
    print(f"✅ Saved {saved_non_defective_images} non-defective images to: {model1_output_folder}")

ℹ️ Found 61 images in input folder for Model 1.
=== Running Model 1: Teeth Detection ===

image 1/1 D:\ai\subhi\undistorted gear images\1.jpg: 384x640 1 spur gear, 20 teeths, 91.8ms
Speed: 11.4ms preprocess, 91.8ms inference, 342.4ms postprocess per image at shape (1, 3, 384, 640)
❌ 1.jpg: Defective teeth count (21) != expected (18)

image 1/1 D:\ai\subhi\undistorted gear images\10.jpg: 384x640 1 spur gear, 20 teeths, 8.8ms
Speed: 5.3ms preprocess, 8.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
❌ 10.jpg: Defective teeth count (21) != expected (18)

image 1/1 D:\ai\subhi\undistorted gear images\11.jpg: 384x640 1 spur gear, 20 teeths, 9.2ms
Speed: 3.0ms preprocess, 9.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
❌ 11.jpg: Defective teeth count (21) != expected (18)

image 1/1 D:\ai\subhi\undistorted gear images\12.jpg: 384x640 1 spur gear, 20 teeths, 15.6ms
Speed: 3.0ms preprocess, 15.6ms inference, 5.3ms postprocess per image at shape (1, 3

In [25]:
print("=== Running Model 2: Circle Detection & Diameter Measurement ===")
for fname in os.listdir(model1_output_folder):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(model1_output_folder, fname)
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Cannot read image {fname}, skipping.")
        continue

    results = model2(img_path)[0]

    found_circle = False
    annotated_img = img.copy()

    if results.boxes and results.boxes.xyxy is not None and len(results.boxes.xyxy) > 0:
        for box in results.boxes.xyxy.cpu().numpy():
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            radius = min((x2 - x1), (y2 - y1)) // 2
            diameter_px = radius * 2

            raw_diameter_mm = diameter_px * PIXEL_TO_MM
            corrected_diameter_mm = raw_diameter_mm + ERROR_CORRECTION_MM

            if abs(corrected_diameter_mm - EXPECTED_DIAMETER_MM) <= TOLERANCE_MM:
                status = "✅ OK"
                color = (0, 255, 0)
            else:
                status = "❌ DEFECTIVE"
                color = (0, 0, 255)

            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
            cv2.circle(annotated_img, (cx, cy), radius, color, 2)
            cv2.putText(annotated_img, status, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            diameter_log.append(
                f"{fname}: Center=({cx},{cy}), Radius={radius}px, "
                f"Diameter={diameter_px}px, Raw={raw_diameter_mm:.2f}mm, "
                f"Corrected={corrected_diameter_mm:.2f}mm, Status={status}"
            )

            found_circle = True
            break  # Assuming only one main circle to detect
    else:
        print(f"⚠️ No circle detected in {fname}")

    annotated_path = os.path.join(model2_annotated_folder, fname)
    saved_annotated = cv2.imwrite(annotated_path, annotated_img)
    if not saved_annotated:
        print(f"⚠️ Failed to save annotated image {annotated_path}")

    if found_circle:
        saved_circle = cv2.imwrite(os.path.join(model2_circle_folder, fname), annotated_img)
        if not saved_circle:
            print(f"⚠️ Failed to save circle image for {fname}")

with open(log_path, "w") as f:
    f.write("\n".join(diameter_log))

print(f"\n✅ Pipeline finished.\n- Non-defective teeth images: {model1_output_folder}\n- Annotated circle images: {model2_annotated_folder}\n- Circle images: {model2_circle_folder}\n- Diameter log saved at: {log_path}")

=== Running Model 2: Circle Detection & Diameter Measurement ===

✅ Pipeline finished.
- Non-defective teeth images: D:\ai\subhi\predictions\model1_non_defective
- Annotated circle images: D:\ai\subhi\predictions\model2_annotated
- Circle images: D:\ai\subhi\predictions\model2_annotated\circles
- Diameter log saved at: D:\ai\subhi\predictions\diameter_log.txt
